# <b><font color="pink" size="+2"> Código creado por Cagliostro

Última actualización: 04/03/2023.

Agradecimientos a:

-NVIDIA, por crear la tecnología [Tacotron](https://github.com/NVIDIA/tacotron2).

-La comunidad de [FakeYou](https://) en español.

-Ediciones Pakirri, *sciunt qui sunt*.

## **1** Preparamos el cuaderno

In [ ]:
#@markdown ## **1.1** Descargamos las dependencias

!pip install pydub
!pip install numpy
!pip install yt-dlp
!mkdir audios
!mkdir pinkfloyd
!mkdir resultado

In [ ]:
#@markdown ## **1.2** Descargamos el temita de Pink Floyd jamón y se guarda en la carpeta (/content/pinkfloyd)

!yt-dlp -f 'bestaudio[ext=mp3]/best[ext=mp3]/bestaudio/best' -x --audio-format mp3 --audio-quality 320K https://www.youtube.com/watch?v=cWGE9Gi0bB0 -o '/content/pinkfloyd/background.%(ext)s'


## **2** Ahora hay que subir los audios que quieras unir a la carpeta audios (/content/audios)

In [ ]:
#@markdown ## **2.1** Subimos los archivos
from google.colab import files
import os

# Abrir cuadro de diálogo para seleccionar archivos
uploaded = files.upload()

# Mover los archivos seleccionados a la carpeta "/content/audios"
for filename in uploaded.keys():
  source_path = filename
  destination_path = os.path.join('/content/audios/', filename)
  os.rename(source_path, destination_path)


Saving descarga.wav to descarga.wav


## **3** Acabamos

In [ ]:
#@markdown ## **3.1** Unimos los audios
import os
import librosa
import soundfile as sf
from pydub import AudioSegment
import numpy as np

# Ruta a la carpeta con los archivos de audio
audio_folder = "/content/audios/"

# Obtener la lista de archivos de audio en la carpeta y ordenarlos alfanuméricamente
audio_files = os.listdir(audio_folder)
audio_files.sort()

# Cargar cada archivo de audio y concatenarlos en uno solo con 600ms de silencio
concatenated_audio = None
for file in audio_files:
    # Cargar el archivo de audio
    audio, sr = librosa.load(os.path.join(audio_folder, file), sr=None, mono=False)

    # Agregar 600ms de silencio al final del archivo de audio
    silence_duration = 0.6  # Duración del silencio en segundos
    num_samples_silence = int(silence_duration * sr)
    silence = np.zeros(num_samples_silence)
    audio_with_silence = np.concatenate((audio, silence), axis=0)

    # Concatenar los archivos
    if concatenated_audio is None:
        concatenated_audio = audio_with_silence
    else:
        concatenated_audio = np.concatenate((concatenated_audio, audio_with_silence), axis=0)

# Calcular la duración total del audio resultante
duration = librosa.get_duration(y=concatenated_audio, sr=sr)

# Ruta a la canción de Pink Floyd
song_path = "/content/pinkfloyd/background.mp3"

# Cargar la canción y ajustar su duración a la duración del audio resultante
song = AudioSegment.from_file(song_path, format="mp3")
song = song[:int(duration*1000)]
song = song - 2 # Reducir el volumen en 2 dB

# Convertir el audio concatenado en un archivo de sonido y guardarlo en disco
sf.write("/content/resultado/final.wav", concatenated_audio.T, sr)

# Abrir el archivo de sonido y agregar la canción de Pink Floyd en segundo plano
final_audio = AudioSegment.from_wav("/content/resultado/final.wav")
final_audio = final_audio.overlay(song, loop=True)

# Guardar el audio final en disco
final_audio.export("/content/resultado/final_con_cancion.mp3", format="mp3")


In [ ]:
#@markdown ## **3.2** Descargamos el audio final
from google.colab import files

# Descargar archivo
files.download('/content/resultado/final_con_cancion.mp3')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>